# Lab #5 Report

**Group Members**: Tyrese, Liam

**Class**: Physics

**Originally Submitted**: October 13, 2022

## Problem Statement
* In this lab, we are asked to extend the work from Lab #2 by simulating a binary system in which the Earth and the Sun have comparable masses. This adjustment allows us to model two celestial bodies orbiting each other, similar to a dual-star system.
* The simulation must incorporate visual representations of both net force vectors and momentum vectors for each body. Specifically, the Earth and the Sun should each display two arrows: one representing momentum and one representing net force.
* Additionally, the simulation must include graphical outputs that display the work and kinetic energy of the system. Individual graphs are required for the work done on the Earth and the work done on the Sun, along with a graph showing the total work of the system. Finally, a graph showing the change in kinetic energy for both objects over time must be produced.

## The Code

In [1]:
from vpython import *

class Dummy():
    pass

def run_simulation(show_simulation: True, earth_work=False, sun_work=False,
                   total_work=False, kinetic_energy=False, print_message=False):
    """
    Run a simplified Sun–Earth orbital simulation and optionally display 
    VPython graphs of work and kinetic energy over time.

    Args:
        show_simulation (bool):
            If True, creates VPython 3D objects (Sun and Earth) and shows their 
            orbital motion.
            
        earth_work (bool):
            If True, plots the work done by the gravitational force on Earth 
            as a function of time.
            
        sun_work (bool):
            If True, plots the work done by the gravitational force on the Sun.
            Default is False.
            
        total_work (bool):
            If True, plots the total work (Sun + Earth). 
            
        kinetic_energy (bool):
            If True, plots the change in kinetic energy of the Sun, Earth, and 
            the combined system.
            
        print_message (bool):
            If True, prints the orbital period of the Earth in days once half 
            an orbit is detected.
    """
    
    # Gravitational constant
    G = 6.7e-11

    if show_simulation:
        # Create the "Sun"
        Sun = sphere (pos=vec(-1.1e11, 0, 0),
                      mass = 2e24,
                      radius = 10 * 7e8,
                      v = vec(0, 0, 0),
                      color=color.orange,
                      make_trail = True)
        
        # Create the "Earth"
        Earth = sphere (pos=vec(1.5e11, 0, 0),
                        mass = 1e24,
                        radius = 6 * 6.4e8,
                        v = vec(0, 1e1,0),
                        color=color.blue,
                        make_trail = True)
    else:
        # Use dummy objects to store attributes without showing graphics
        Sun = Dummy()
        Sun.pos = vec(-1.1e11, 0, 0)
        Sun.mass = 2e24
        Sun.v = vec(0, 0, 0)
        
        Earth = Dummy()
        Earth.pos = vec(1.5e11, 0, 0)
        Earth.mass = 1e24
        Earth.v = vec(0, 1e1,0)

    # Calculate initial distance vector from Sun to Earth
    r = Earth.pos - Sun.pos
    
    # Initial gravitational force (Newton’s law of gravitation)
    F = - G * Sun.mass * Earth.mass / r.mag**2 * r.hat
    
    # Initialize momentum of Earth and Sun (p = m*v)
    Earth.p = Earth.mass * Earth.v
    Sun.p = -Earth.p
    
    # Assign forces for visualization (arrows)
    Earth.f = F
    Sun.f = -F

    if show_simulation:
        # Attach arrows showing force (f) and momentum (p) for Earth and Sun
        myarrowfe = attach_arrow(Earth, "f", scale=1e-6, color=color.blue)
        myarrowfs = attach_arrow(Sun, "f", scale=1e-6, color=color.yellow)
        myarrowpe = attach_arrow(Earth, "p", scale=1e-15, color=color.green)
        myarrowps = attach_arrow(Sun, "p", scale=1e-15, color=color.orange)
    
    # Initial kinetic energy of each body and total
    KE_01 = Sun.p.mag2 / (2 * Sun.mass)
    KE_02 = Earth.p.mag2 / (2 * Earth.mass)
    KE_0 = Earth.p.mag2 / (2 * Earth.mass) + Sun.p.mag2/(2*Sun.mass)
    
    # Time variables
    t = 0
    dt = 14 * 24 * 60 * 60  # timestep = 2 weeks/sec

    # Work variables
    work_e = 0
    work_s = 0
    work_c = 0

    # Graphs only when requested
    if earth_work:
        work_graph_e = graph(title = ' Earth Work ', xtitle = ' t ', ytitle = ' W ')
        w_e = gcurve(graph = work_graph_e, color = color.blue)
    if sun_work:
        work_graph_s = graph(title = ' Sun Work ', xtitle = ' t ', ytitle = ' W ')
        w_s = gcurve(graph = work_graph_s, color = color.yellow)
    if total_work:
        work_graph_c = graph(title = ' Total Work ', xtitle = ' t ', ytitle = ' W ')
        w_c = gcurve(graph = work_graph_c, color = color.green)
    if kinetic_energy:
        ke_graph = graph(title = ' Change in Kinetic Energy ', xtitle = ' t ', ytitle = ' E ')
        DK1 = gcurve(graph = ke_graph, color = color.red)
        DK2 = gcurve(graph = ke_graph, color = color.blue)
        DK12 = gcurve(graph = ke_graph, color = color.orange)
    
    # Track old momentum and orbital period for Earth
    Earth.old_p = Earth.p 
    Earth.T = 0 # keeps track of how long Earth orbit taking
    
    myrate = 1500 # sets rate of program
    
    # Main simulation loop
    while True:
        rate(myrate)
        
        # Update distance between Earth and Sun
        r = Earth.pos - Sun.pos
    
        # Gravitational force (magnitude & direction)
        F = - G * Earth.mass * Sun.mass / r.mag**2 * r.hat
    
        # Update forces on each body
        Earth.f = F
        Sun.f = -F
    
        # Update momentum using Impulse-Momentum theorem (p_new = p_old + F*dt)
        Earth.p = Earth.p + F * dt
        Sun.p = Sun.p - F * dt
    
        # Update velocities (v = p/m)
        Earth.v = Earth.p / Earth.mass
        Sun.v = Sun.p / Sun.mass
    
        # Update positions (x_new = x_old + v*dt)
        Earth.pos = Earth.pos + Earth.v * dt
        Sun.pos = Sun.pos + Sun.v * dt 
    
        # Work done by forces (W = ∫F·v dt)
        work_e += Earth.f.dot(Earth.v) * dt
        work_s += Sun.f.dot(Sun.v) * dt
        Earth.work = work_e
        Sun.work = work_s
        
        work_c = work_s + work_e

        # Plot work values
        if earth_work: w_e.plot(pos=(t,Earth.work))
        if sun_work:   w_s.plot(pos=(t,Sun.work))
        if total_work: w_c.plot(pos=(t,work_c))
        
        if kinetic_energy:
            # Plot kinetic energy changes (ΔKE = KE - KE_initial
            KE = Sun.p.mag2 / (2 * Sun.mass) + Earth.p.mag2 / (2 * Earth.mass)
            KE1 = Sun.p.mag2 / (2 * Sun.mass)
            KE2 = Earth.p.mag2 / (2 * Earth.mass)

            DK1.plot(pos=(t, KE1 - KE_01))
            DK2.plot(pos=(t, KE2 - KE_02))
            DK12.plot(pos=(t, KE - KE_0))
    
        # Update time
        t += dt
    
        # Detect when Earth completes a half-orbit (momentum x-component changes sign)
        if Earth.old_p.x > 0 and Earth.p.x < 0:
            if print_message:
                print(f"Year: {Earth.T / (60 * 60 * 24)} days")
            else:
                break
    
        # Update Earth’s orbital tracking
        Earth.old_p = Earth.p
        Earth.T = Earth.T + dt
        

/home/gillty/Physic_Labs/jupyter_env/lib/python3.12/site-packages/vpython/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


<IPython.core.display.Javascript object>

## How the Code Solves the Problem
&emsp;&emsp;To obtain the results that we did, we first referred back to our previous lab pertaining to gravitational force between two celestial bodies. However, for the purposes of this week’s lab, we modified the masses of the Earth and Sun respectively to be roughly equivalent to mimic a binary system run. Additionally, we implemented force vectors to be visible throughout the simulation. In particular, we first modified the Earth’s initial velocity to be smaller due to the change in mass. The change was from vector $v = (0, 1.0 \times 2.978e4, 0)$ m/s to $v = (0, 1e1, 0)$ m/s. Otherwise, the Earth along with the Sun appear vertical in terms of their path and do not give us an accurate output of their motion on the plot. 

&emsp;&emsp;We also set the Sun’s Momentum to be the opposite of the Earth’s Momentum so the system remains stationary. We also set the Force of the Earth, and the Sun to be equal to the Net Force respectively. However, they are set to be equal but opposite to each other, which is related to Newton’s Third Law. Also, this means that the Momentum Principle is essentially related to our problem, with the equation $p = mv$. Additionally, the Net Force is essentially the Gravitational Force with respect to each planet. This is given by the equation: $F_g = \frac{-G * m_{Earth} * M_{Sun}} {r_2}$. This solves for the Force of Gravity. And this of course applies to all directions, so it would be multiplied by the unit vector for the sake of our problem. Also $G$ is the gravitational constant, where, $G = 6.6743 \times 10^{-11} \frac{m^3}{kg*s^2}$. Additionally the distance between the objects in motion is represented by $r$. So $r = pos_{Earth} - pos_{Sun}$, where $pos$ represents the position of each. So $r$ represents the change in position, so that it is updated, along with everything else.

&emsp;&emsp;Moreover, we calculated the initial kinetic energies for the Sun, and Earth respectively, and then added the total of them together. This is done outside the while loop, as in order to find the Kinetic Energy, we need the initial to not be updated, but we need the final kinetic energy to be updated as time moves along within the while loop.

&emsp;&emsp;Furthermore, we wrote the code pertaining to the vectors for force and momentum using the scalar values 1e-6 and 1e-15 respectively. Doing so makes observation of the simulation easy to follow with no need to zoom in or out due to the length of the vectors. And we also updated the time step to be large so that it would speed up the simulation to a reasonable pace so that it would become observable. To follow this up, the force of the Sun, and Earth are again put to be equal but opposite to each other to the Net Force in the while loop so that the Net Force arrows for each will be updated continuously.

&emsp;&emsp;The work of the earth, and sun are updated using the Force dot product of each of their velocities multiplied by our new time step.

&emsp;&emsp;This is represented by the equation: $W = F.dot(v) \times dt$. Then we assigned a variable to the total work, where the updated work of the sun, and earth are added together. Then the individual Work Plots (Sun and Earth separately) are initialized with respect to time. Then the final Kinetic Energies for the Sun, Earth, and the combined Kinetic Energy are calculated. The equation: $KE = \frac{P^2}{2 \times m}$ is applied here. This is also applied to the initial Kinetic Energies. This is simply a rewrite of the equation: $KE = \frac{1}{2} mv^2$, which is the principle of Kinetic Energy. To derive this, we can recall that $P = mv$. If we start from the equation we used, we can derive the principle of Kinetic Energy as proof that they’re equivalent. Therefore, $KE = \frac{mv^2} {2m}$. Then $KE = \frac{m^2v^2} {2m}$ since we can move the exponent on the inside. As a result, we can cancel out the mass on the bottom to get: $KE = \frac {mv^2} {2}$. And this will be equivalent to the Kinetic Energy Principle, which is: $KE = \frac{1}{2}mv^2$. What's more, it’s important to note that Newton’s Method is applied here since the variables are updated throughout the time the simulation runs for.


### Simulation Model
![Simulation](assets/gifs/Binary_System.gif)

In [2]:
simulation_model = canvas()
run_simulation(show_simulation=True)

<IPython.core.display.Javascript object>

&emsp;&emsp;As displayed above, as two stars enclose on one another during orbit, their force vectors extend and meet one another while their momentum vectors point in the direction in which each star is traveling. This is to be expected since the closer two objects get, the stronger the  gravitational force between them. This is validated by the location of the variable $r$ which is representative of the distance between the Earth and the Sun. Given that this variable is in the denominator, as the distance $r$ increases, the overall value of the gravitational force will decrease. Thus, as the two stars approach one another and close the distance between them, the gravity will become increasingly stronger, causing them to accelerate towards one another, until they pass and build up distance between them once more and causing the gravitational force between them to dissipate. Moreover, it makes sense that the force of both celestial objects would point towards one another due to the fact that the origin of their increasing gravitational force is each other.

&emsp;&emsp;As for the momentum vectors, they behave as expected as well. Firstly, both vectors point in the direction according to their current path of travel. Given that momentum is equivalent to mass times velocity, this is straightforward to understand since mass is a scalar quantity whereas velocity is speed with direction. This means that the direction of the momentum vector is being derived from the vector quantity, velocity, whose direction is dependent on the direction in which the object is moving. Additionally, we also observed the momentum vectors growing in magnitude as the two celestial bodies grew closer to one another. This is understandable because the velocity of the two bodies will increase due to the formula for the change in momentum which incorporates the variable for gravitational force. As previously mentioned, the gravitational force will increase as the distance between the two stars decreases. With this in mind, the overall momentum will also increase  as the stars near each other as a result of adding (Note: The force of Sun is negative and thus becomes positive from the subtraction sign in the formula for calculating the Sun’s momentum) the increased change in momentum which uses the formula: $F \times \Delta t$.

&emsp;&emsp;Lastly, the momentum and force for the Earth and Sun respectively are equivalent in magnitude and yet opposite in direction. This behaves as expected since we define the positive force and momentum of the Earth and their negative counterparts for the Sun. As a result, the magnitude wouldn’t change since the scalar quantity itself is neither increased nor decreased (plus the fact that length of a vector cannot be negative). However, vector quantity would be affected due to switching the positivity of the vector components which in turn would flip the direction of the vector. This all makes sense because the direction of travel of the Earth and Sun are opposite to one another.


### Graph #1 - Earth Work

In [3]:
earth_work_graph = canvas()

run_simulation(show_simulation=False, earth_work=True)

<IPython.core.display.Javascript object>

&emsp;&emsp;As seen with the graph above, the work of the Earth follows a positive pattern in which it repeatedly transitions between increasing and decreasing whilst its maximum and minimum values gradually increase. To start, the fact that the plot is positive shouldn’t come as no surprise to one already familiar with the concept of work. This is because work is positive whenever the direction of the force applied and the displacement are the same. This can be explained further by taking a look at the formula for work: $Work = Force \times Displacement \times cos(𝜃)$, where 𝜃 is the angle between force and displacement. However, the equation we used for our problem is $Work = F.dot(v) \times dt$. So, in our code we applied the dot product in order to achieve an equivalent result. When the difference in angle between force and displacement is in the range of 0 to 90 degrees or 270 to 360 degrees, the work is positive given that this is where the cosine is positive. Otherwise, the work is negative. This applies to our simulation since the gravitational force acting on the Earth is always pointing in the direction of the Sun. Meanwhile, the direction of displacement is always pointing from the initial to the final position. The work drastically increases, and peaks when the Earth, and the Sun cross paths. This makes sense because the Force of attraction is at its highest when they’re closer together. And when they’re away from each other, the Force is low due to the fact that the distance between them is greater. This makes sense by the equation, $F_g = \frac {G \times M_{Sun} \times m_{Earth}} {r^2}$. The Net Force is inversely proportional to the square of the distance between them. And the Force is directly proportional to the Work, as given by the previous equation. So, as the distance between them increases, the Force of Gravity decreases by the inverse square proportion. Therefore, as the Force decreases due to the increase in distance between them, the work decreases by direct proportion using the Work equation. And the opposite will be true. So, as the distance between them decreases, the Force of Gravity will increase, and then as a result, the Work will increase when they get close together. And the same basic idea applies to the rest of the work graphs, in terms of the Sun individually, and the combined Earth & Sun work graphs.

&emsp;&emsp;And the graph also lines up with what’s going on in our simulation. When the Work increases, we see the arrows of the Force vectors increasing when they get closer together. And this increases rapidly when they almost come into contact. And when they get further away from each other, the Force arrows are smaller, and this is shown in the Work graph, which depends on the Force, in how the Work is quite small as well. So the proportional relationships in the equations showcase what’s going on in the simulation, along with the graph plotted.
    
&emsp;&emsp;Let's visualize when the Sun and Earth are directly across from one another at their initial starting points: The displacement vector of the Earth will be pointing upwards while the force vector will be pointing leftwards towards the Force of Gravity produced from the Sun. The difference between up (90°) and left (180°) is about 90° and the cosine of 90 ° is 0. This value represents the first plot on our graph.

&emsp;&emsp;The work is then shown to increase dramatically. For context, this happens each time the Earth and the Sun pass their initial starting points and approach one another. Thus, such a result makes sense given that the gravitational force and velocity incorporated into the work formula will increase as the distance between the two celestial bodies shrinks and decreases the value of the denominator for the overall force equation, thereby increasing the force which in turn increases velocity given the formulas for momentum and velocity respectively. Therefore, as the gravitational force increases, so does the work of the Earth.

&emsp;&emsp;Moving on, after the Earth and Sun have passed each other, the graph for work follows a decreasing trend up until they reach their former starting points in the simulation. This was to be expected since the two bodies would begin to experience a decrease in velocity during this period due to the distance between them growing as they continued down their orbital pathway away from one another. To explain this scientifically, the decrease in velocity can be attributed to momentum, which is the numerator in the formula for velocity as shown: $Velocity = \frac {Momentum} {Mass}$. The formula for momentum includes force which we explained previously to be decreasing during this part of the simulation due to the decreasing denominator component, $r$.

&emsp;&emsp;Lastly, as time progresses, the graph for work takes a “step” upwards – increasing the value of both its maximum and minimum following each dip. This makes sense since with each run of the while loop, we add onto the work for Earth. Doing this doesn’t affect when the graph for work will increase or decrease since that is determined by the variable components of work. Instead, this shifts the values upwards every time the while loop runs. Consequently, this leads to the development of the graph that we obtained.

### Graph #2 - Sun Work

In [4]:
sun_work_graph = canvas()

run_simulation(show_simulation=False, sun_work=True)

<IPython.core.display.Javascript object>

&emsp;&emsp;In the case of the graph displaying the work for the Sun, we can see the same general trends as were previously explained in that for the Earth. As with the work for Earth, the graph is positive due to the direction of the force and displacement being in the same general direction throughout the simulation which leads to the cosine of theta becoming a positive number. Additionally, the graph starts at 0 due to the right angle that initially forms at the start of the simulation between the force vector (which points rightward at an angle of 0° or 360°) and the displacement vector (which points downward at an angle of 270°). Taking into account that the cosine of 90° is 0, this would make the overall value for work be equivalent to 0 at the beginning of the graph (which it is).

&emsp;&emsp;Furthermore, the constantly changing distance between the Sun and the Earth due to their overlapping orbital pathways is responsible for the sudden drops and rises present throughout the graph. This is because as they get closer, the gravitational force between them becomes stronger due to the decreasing variable $r$ in the denominator which represents the distance between the two celestial bodies. In turn, this increased force increases velocity via the momentum principle – causing both celestial bodies to experience an increase in gravitational force and increase in velocity simultaneously as is shown in the simulation. On the other hand, after they pass one another and part ways, the distance increases which in turn increases the denominator for the force formula and decreases the gravitational force between the two “stars”. Altogether, this explains the behavior of the graph, with it constantly switching from increasing to decreasing as the two celestial bodies constantly move closer to each other and then away from one another during their orbits.

&emsp;&emsp;Moreover, as with the “Earth Work” graph, the graph representative of the work of the Sun seemingly takes a “step” upwards. As with the Earth, this is the result of adding the new value of work to the initial value. This causes the values of the plot to constantly shift upward whilst not impacting the when or where the graph increases and decreases. In turn, this creates a sort of “staircase” as the work constantly builds up with each run of the simulation.

&emsp;&emsp;A distinct characteristic to note that distinguishes the “Earth Work” graph from the “Sun Work” graph is the fact that the work for the earth is represented with values to the power of 27 whereas those of the sun are to the power of 26. A thoughtful investigation into this conundrum has led us to the conclusion that this is because of the Sun’s larger mass. Consider the formula: Velocity = Momentum / Mass. An important thing to note is that the overall value of the velocity will decrease when the denominator, mass, increases. Given that the mass of the Sun is 2e24 kg while the mass of the Earth is 1e24 kg, it should be expected that the overall velocity of the Sun will be lower than that of the Earth. This falls in line with what we see in the simulation where we observe that the Sun with its smaller orbital ring completes a single orbit at the same time as its celestial neighbor which possesses a longer orbital pathway, thereby implying that the Sun does in fact have a slower velocity. In turn, when we calculate the work of the Sun, we can expect a smaller yield in comparison to the work of the Earth. 

### Graph #3 - Total Work

In [5]:
total_work_graph = canvas()

run_simulation(show_simulation=False, total_work=True)

<IPython.core.display.Javascript object>

&emsp;&emsp;As shown above, the graph for “Total Work” displays an increasing-decreasing trend initialized at 0 and making higher “jumps” as the simulation progresses. This makes sense since we are simply finding the sum of the work values of the Earth and Sun. Considering that the behavior of the work graphs for both the Earth and Sun respectively are essentially mirroring each other, we would expect the same trends to be inherited if we were to add the values from both together. Additionally, the range of values from the “Total Work” is larger in comparison to the graphs for the Earth and Sun, with a maximum y-value of 2.0e27 and 10e26 (1.0e27) respectively. In contrast, the graph above has a maximum y-value of 3.0e27 which is equivalent to the sum: 2.0e27 + 1.0e27. This is to be expected considering that we are adding together the values, thus leading to such quantities.

### Graph #4 - Change in Kinetic Energy

In [6]:
change_in_kinetic_energy_graph = canvas()

run_simulation(show_simulation=False, kinetic_energy=True)

<IPython.core.display.Javascript object>

&emsp;&emsp;The graph above for “Change in Kinetic Energy” shows three lines: red for the change of the kinetic energy of the sun, blue for the change of kinetic energy of the Earth, and orange for the combined change of kinetic energy of the Sun and Earth. In essence, we see a reflection of our Work graphs prior. This also reflects the equation $W = ΔKE$, which is the Work & Kinetic Energy Theorem. It states that the Work is equal to the change in kinetic energy. In order to explain this graph, we see that, like the Net Work graph, the total change in kinetic energy (both the Sun, and the Earth), peaks at 2.5e27 Joules, just like the "Total Work" graph does in its first iteration when the path of the objects in motion overlap. This is the same for the Earth, and Sun’s change in Kinetic Energy. While the Earth’s change in kinetic energy peaks at approximately 1.7e27 Joules, the Earth’s Work also peaks at approximately peaks at the same value of Joules. And this is also true for the Sun’s change in kinetic energy, where it peaks at about 0.85e27 Joules, which can be rewritten as 8.5e26 Joules. And on the graph for the "Sun Work", on its first iteration, it approximately peaks at 8.5e26 Joules as well. So, by the equation, we get what we expect. However, it’s important to note, as we explained earlier, that the Work graphs essentially gradually lift up as time moves on. This is because the Work is added to the original Work done over time. Therefore, we see this reflected in the Work graphs. However, the range between the values of the Work remains the same, which further proves our "Change in Kinetic Energy" graph, as the range between the maximum and minimum in the Work values remains constant, and equal in each iteration. So our Work Graphs, and Kinetic Energy graphs, reflect each other, and prove the Work & Kinetic Energy Theorem, as shown by $W = ΔKE$.

&emsp;&emsp;Additionally, the "Change in Kinetic Energy" graph reflects our simulation by the equation used to obtain the values. The equation used is essentially: $ΔKE = \frac {P_f^2}{2m} - \frac {p_i^2}{2m}$. And as we proved earlier, this is equal to the fundamental Kinetic Energy equation, $KE = \frac{1}{2}mv^2$. This means we can rewrite this as $ΔKE = \frac {1}{2} mv_f^2 - \frac {1}{2}mv_i^2$. The right side of the equation, as shown by the initial Momentum remains constant for our simulation since we did one run, as the initial velocity does not change. However, the left side of the equation changes over time, so we can use this to prove that our graph relates to the equation utilized in our code.

&emsp;&emsp;The formula $ΔKE = \frac {p_f^2} {2m} - \frac {p_i^2} {2m}$ shows that as the change in kinetic energy increases, the value that changes must be the Final Momentum. And the Final Momentum must increase overall. This is shown by the equation since the change in kinetic energy is proportional to the Final Momentum Squared. Additionally, the initial Momentum is negatively proportional to the change in kinetic energy. This means that the change in kinetic energy will always increase in the negative direction if the initial Momentum decreases. This means that if it approaches 0, it will be decreasing. But, if it is approaching negative infinity, it will have a larger negative change in kinetic energy as a result.

&emsp;&emsp;This also relates to the simulation, as we see the Kinetic Energy peaking when the Momentum vector arrows are drastically increasing. And when the Momentum vector arrows are extremely small, the values on our "Change in Kinetic Energy" graph are also quite small in scale in comparison to the peaks. And this is also when the objects are far away from each other, so the Force between them will be small, and the Momentum responds accordingly. This ultimately achieves what we expect, as it proves that the change in kinetic energy relates to the Work, and matches with what happens in the simulation, as when they’re closer together, the Momentum rapidly increases, as shown by the peaks in the graphs, and is small when the objects in motion are farther away from each other.

## Conclusion

&emsp;&emsp;In this lab we referred to our previous work from Lab #2 and were tasked with implementing force and momentum vectors as well as graphs related to work and kinetic energy. To obtain the results that we did, we modified the masses of the Earth and Sun to be 1e24 and 2e24 respectively, we changed the initial y-component of the Earth’s velocity to be 1e1, we made the momentum and gravitational force of the Sun and Earth be equal but opposite to each other, and more. These modifications ensured the proper simulation of a binary system with easily visible and recognizable features. And of course, we added Momentum, and Force arrows to each object in motion.

&emsp;&emsp;The physics relevant to the problem in question include the momentum principle, gravitational force, work, kinetic energy principle, and the work & kinetic energy theorem. As the Earth and the Sun get closer to one another, the orange and green vectors for momentum increase in length whilst always pointing in the direction at which the object is speeding towards due to velocity. Essentially, the momentum is pointing toward the orbital path the object is moving in. Meanwhile, the vectors for gravitational force shown as blue and yellow vectors increase in length the closer they get to their neighbor as a result of the decreasing denominator, $r$, in the gravitational force formula. Moreover, the gravitational force vectors always point in the direction of their neighboring celestial object which represents the origin of this force. Furthermore, work for both the Earth and the Sun is shown to increase and decrease based on the value of the gravitational force between the Earth and the Sun which itself is based on the distance between these two bodies. Thus, as they approach each other, work increases in value. Vice versa, as they move away from each other, work decreases in value. Lastly, the change in kinetic energy for the Earth and Sun individually, along with the combined change in kinetic energy is shown to increase when velocity increases and decrease when velocity decreases.

### What We Learned
1. How to implement the code pertaining to work in a digital setting.
2. The relationship between gravitational force and displacement with regard to work.
3. How to describe the graphical behavior of work
    - Work increases as the Earth and Sun approach each other; work increases as velocity and gravitational force increase
4. How to better describe the graphical behavior of kinetic energy
    - Kinetic energy increases/decreases with velocity/momentum
5. A larger object doesn’t always necessarily mean a higher work value than a smaller object
    - This was visible with the work graph for the Sun where the Sun’s smaller velocity decreases the overall values for work to the point where it is smaller than the values for work